In [11]:
import struct
import numpy as np
import json

# File path (update if needed)
file_path = "restart_all_GL05RL01z40.pe000000"
output_json = file_path+"_reversed.json"

# Mapping from datatype code to NumPy dtype
dtype_map = {0: "f4", 1: "f8", 2: "i4", 3: "i8"}

with open(file_path, "rb") as f:
    # Read metadata (header + note)
    header = f.read(64).decode(errors="ignore").strip()
    note = f.read(256).decode(errors="ignore").strip()

    # Read global file metadata
    fmode, endiantype, grid_topology, glevel, rlevel, num_of_rgn = struct.unpack(">6I", f.read(4 * 6))
    
    # Read region IDs
    rgnid = struct.unpack(f">{num_of_rgn}I", f.read(4 * num_of_rgn))
    
    # Read number of data variables
    num_of_data = struct.unpack(">I", f.read(4))[0]

    # Dictionary to store extracted data
    dataset = {
        "Header": header,
        "Note": note,
        "File Mode": fmode,
        "Endian Type": endiantype,
        "Grid Topology": grid_topology,
        "Grid Level": glevel,
        "Resolution Level": rlevel,
        "Number of Regions": num_of_rgn,
        "Region IDs": list(rgnid),
        "Number of Data Entries": num_of_data,
        "Variables": {}
    }

    # Expected shape before reordering: (Region, Layer, ij)
    expected_shape = (5, 42, 324)

    # Process each data entry
    for _ in range(num_of_data):
        # Read variable metadata
        varname = f.read(16).decode(errors="ignore").strip()
        description = f.read(64).decode(errors="ignore").strip()
        unit = f.read(16).decode(errors="ignore").strip()
        layername = f.read(16).decode(errors="ignore").strip()
        note = f.read(256).decode(errors="ignore").strip()

        # Read integer fields
        datasize, datatype, _, _ = struct.unpack(">Q3I", f.read(8 + 4 * 3))

        # Read time information
        time_start, time_end = struct.unpack(">QQ", f.read(8 * 2))

        # Read raw data
        raw_data = f.read(datasize)

        # Convert to NumPy array in correct format (big-endian float)
        data_array = np.frombuffer(raw_data, dtype=np.dtype(f">{dtype_map[datatype]}"))

        # Ensure the correct number of elements before reshaping
        if data_array.size != np.prod(expected_shape):
            print(f"Warning: Variable {varname} has unexpected size {data_array.size}. Skipping...")
            continue

        # Reshape into (Region, Layer, ij)
        data_array = data_array.reshape(expected_shape)

        # Reverse dimensions to (ij, Layer, Region)
        data_array = np.transpose(data_array, (2, 1, 0))

        # Store variable data
        dataset["Variables"][varname] = {
            "Description": description,
            "Unit": unit,
            "Layer Name": layername,
            "Time Start": time_start,
            "Time End": time_end,
            "Data": data_array.tolist()  # Convert NumPy array to list for JSON serialization
        }

# Save as JSON file
with open(output_json, "w") as json_file:
    json.dump(dataset, json_file, indent=4)

print(f"JSON file saved as {output_json}")


FileNotFoundError: [Errno 2] No such file or directory: 'restart_all_GL05RL01z40.pe000008'